In [1]:
import pandas as pd

# Load openai

In [72]:
import boto3 
ssm = boto3.client('ssm', region_name='ap-northeast-2')
def get_parameter(name, with_decryption=True):
    """AWS Parameter Store에서 값을 가져오는 함수"""
    return ssm.get_parameter(Name=name, WithDecryption=with_decryption)['Parameter']['Value']

SECRET_KEY = get_parameter('/TEST/CICD/STREAMLIT/OPENAI_API_KEY')

# PDF 업로드

In [12]:
# !pip install -qU pypdf langchain-community sentence-transformers

In [2]:
FILE_PATH = "../data/frontend_resume.pdf"
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")


In [11]:
from langchain_community.document_loaders import PyPDFLoader

# 파일 경로 설정
loader = PyPDFLoader(FILE_PATH)

# PDF 로더 초기화
docs = loader.load()

# 문서의 전체 내용 출력
resume_text = ""
for doc in docs:
    resume_text += doc.page_content + "\n"  # 각 페이지 내용을 결합

# 전체 내용 출력
print(resume_text)

프 론 트 엔 드  개 발 자  이 력 서
1
프론트엔드  개발자  이력서
이름  및  연락처
이름 : 홍길동
이메일 : hong.gildong@example.com
전화번호 : +82 10-1234-5678
GitHub: https://github.com/honggildong
포트폴리오 : https://honggildong.dev
자기소개
사용자  친화적인  웹  인터페이스  개발에  열정을  가진  신입  프론트엔드  개발자입니다 .
모던  자바스크립트  프레임워크를  활용하여  가치  있는  프로젝트에  기여하고  싶습니다 .
기술  스택
언어: HTML, CSS, JavaScript, TypeScript
프레임워크 / 라이브러리: React, Vue.js, Next.js
도구: Git, Webpack, Figma
기타: 반응형  디자인 , RESTful API
프로젝트  경험
1. 포트폴리오  웹사이트
기간: 2023 년  3 월  - 2023 년  4 월
기술  스택: React, styled-components, GitHub Pages
설명: 개인  프로젝트로 , React 와  styled-components 를  사용하여  포트폴리오  웹사이
트를  제작
주요  기여:
프 론 트 엔 드  개 발 자  이 력 서
2
반응형  디자인을  적용하여  다양한  디바이스에서  최적화된  사용자  경험  제공
애니메이션  효과  (e.g., 페이지  전환 , 컴포넌트  로딩  애니메이션 ) 구현
GitHub Pages 를  활용한  배포  및  CI/CD 를  설정하여  빠른  업데이트  가능
성과: 포트폴리오를  통해  면접  요청  2 건  수락
2. 이커머스  웹  애플리케이션
기간: 2023 년  1 월  - 2023 년  3 월
기술  스택: Vue.js, Vuex, Firebase, Stripe API
설명: 팀  프로젝트로  이커머스  웹  애플리케이션  개발
주요  기여:
Vuex 를  활용한  상태  관리  구현  및  효율적인  데이터 

In [15]:
def preprocess_text(text):
    lines = text.split('\n')
    processed_lines = [line.strip() for line in lines if line.strip()]
    return "\n".join(processed_lines)

cleaned_text = preprocess_text(resume_text)
print(cleaned_text)

프 론 트 엔 드  개 발 자  이 력 서
1
프론트엔드  개발자  이력서
이름  및  연락처
이름 : 홍길동
이메일 : hong.gildong@example.com
전화번호 : +82 10-1234-5678
GitHub: https://github.com/honggildong
포트폴리오 : https://honggildong.dev
자기소개
사용자  친화적인  웹  인터페이스  개발에  열정을  가진  신입  프론트엔드  개발자입니다 .
모던  자바스크립트  프레임워크를  활용하여  가치  있는  프로젝트에  기여하고  싶습니다 .
기술  스택
언어: HTML, CSS, JavaScript, TypeScript
프레임워크 / 라이브러리: React, Vue.js, Next.js
도구: Git, Webpack, Figma
기타: 반응형  디자인 , RESTful API
프로젝트  경험
1. 포트폴리오  웹사이트
기간: 2023 년  3 월  - 2023 년  4 월
기술  스택: React, styled-components, GitHub Pages
설명: 개인  프로젝트로 , React 와  styled-components 를  사용하여  포트폴리오  웹사이
트를  제작
주요  기여:
프 론 트 엔 드  개 발 자  이 력 서
2
반응형  디자인을  적용하여  다양한  디바이스에서  최적화된  사용자  경험  제공
애니메이션  효과  (e.g., 페이지  전환 , 컴포넌트  로딩  애니메이션 ) 구현
GitHub Pages 를  활용한  배포  및  CI/CD 를  설정하여  빠른  업데이트  가능
성과: 포트폴리오를  통해  면접  요청  2 건  수락
2. 이커머스  웹  애플리케이션
기간: 2023 년  1 월  - 2023 년  3 월
기술  스택: Vue.js, Vuex, Firebase, Stripe API
설명: 팀  프로젝트로  이커머스  웹  애플리케이션  개발
주요  기여:
Vuex 를  활용한  상태  관리  구현  및  효율적인  데이터 

In [16]:
type(cleaned_text)

str

In [74]:
import pdfplumber

def extract_project_section(pdf_path):
    # PDF 텍스트 추출
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ''
        for page in pdf.pages:
            full_text += page.extract_text()
    
    # "프로젝트 경험" 섹션 추출
    start_keyword = "프로젝트 경험"
    end_keywords = ["학력", "자기소개"]  # 다음 섹션의 시작 키워드
    project_text = ""

    # "프로젝트 경험" 시작 위치 찾기
    if start_keyword in full_text:
        start_index = full_text.find(start_keyword)
        project_text = full_text[start_index:]

        # "프로젝트 경험" 이후 섹션 제거
        for end_keyword in end_keywords:
            if end_keyword in project_text:
                end_index = project_text.find(end_keyword)
                project_text = project_text[:end_index]
                break

    return project_text.strip()

# 사용 예시
project_section = extract_project_section(FILE_PATH)
print("프로젝트 경험 섹션:\n", project_section)

프로젝트 경험 섹션:
 프로젝트 경험
포트폴리오 웹사이트
1.
기간: 2023년 3월 - 2023년 4월
기술 스택: React, styled-components, GitHub Pages
설명: 개인 프로젝트로, React와 styled-components를 사용하여 포트폴리오 웹사이
트를 제작
주요 기여:
프론트엔드 개발자 이력서 1반응형 디자인을 적용하여 다양한 디바이스에서 최적화된 사용자 경험 제공
애니메이션 효과 (e.g., 페이지 전환, 컴포넌트 로딩 애니메이션) 구현
GitHub Pages를 활용한 배포 및 CI/CD를 설정하여 빠른 업데이트 가능
성과: 포트폴리오를 통해 면접 요청 2건 수락
이커머스 웹 애플리케이션
2.
기간: 2023년 1월 - 2023년 3월
기술 스택: Vue.js, Vuex, Firebase, Stripe API
설명: 팀 프로젝트로 이커머스 웹 애플리케이션 개발
주요 기여:
Vuex를 활용한 상태 관리 구현 및 효율적인 데이터 흐름 설계
Stripe API를 사용해 결제 기능 통합 및 테스트 환경 구축
제품 카테고리별 필터링 및 검색 기능 개발
Firebase Authentication을 활용한 사용자 인증 및 권한 관리 추가
성과: 프로젝트 완료 후 80% 이상의 코드 리뷰 반영
블로그 플랫폼 개발
3.
기간: 2022년 10월 - 2022년 12월
기술 스택: Next.js, Markdown, Tailwind CSS, Vercel
설명: Markdown 파일 기반의 정적 블로그 플랫폼 개발
주요 기여:
Next.js의 정적 사이트 생성(SSG) 기능을 활용하여 빠르고 SEO 친화적인 페이지
구현
Tailwind CSS를 사용하여 간결한 UI/UX 디자인 구축
다크 모드 및 커스텀 테마 옵션 추가
포스트 태그 관리 기능 및 태그 기반 검색 기능 구현
성과: 개인 블로그 운영으로 월간 방문자 500명 기록
실시간 채팅 애플리케이션
4.
프론트엔드 개발자 이력서 2기간: 2022년 6월 - 2022년 8월
기술 스택: Re

In [69]:
import pdfplumber

def extract_project_section(pdf_path):
    # PDF 텍스트 추출
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ''
        for page in pdf.pages:
            full_text += page.extract_text()
    
    # "프로젝트 경험" 섹션 추출
    start_keyword = "프로젝트 경험"
    end_keywords = ["학력", "자기소개"]  # 다음 섹션의 시작 키워드
    project_text = ""

    # "프로젝트 경험" 시작 위치 찾기
    if start_keyword in full_text:
        start_index = full_text.find(start_keyword)
        project_text = full_text[start_index:]

        # "프로젝트 경험" 이후 섹션 제거
        for end_keyword in end_keywords:
            if end_keyword in project_text:
                end_index = project_text.find(end_keyword)
                project_text = project_text[:end_index]
                break

    return project_text.strip()

# 사용 예시
project_section = extract_project_section(FILE_PATH)

# 1000자 단위로 분할
chunk_size = 1000
project_chunks = [project_section[i:i + chunk_size] for i in range(0, len(project_section), chunk_size)]

# 분할된 내용 출력
for idx, chunk in enumerate(project_chunks):
    print(f"Chunk {idx + 1}:\n{chunk}\n")

Chunk 1:
프로젝트 경험
포트폴리오 웹사이트
1.
기간: 2023년 3월 - 2023년 4월
기술 스택: React, styled-components, GitHub Pages
설명: 개인 프로젝트로, React와 styled-components를 사용하여 포트폴리오 웹사이
트를 제작
주요 기여:
프론트엔드 개발자 이력서 1반응형 디자인을 적용하여 다양한 디바이스에서 최적화된 사용자 경험 제공
애니메이션 효과 (e.g., 페이지 전환, 컴포넌트 로딩 애니메이션) 구현
GitHub Pages를 활용한 배포 및 CI/CD를 설정하여 빠른 업데이트 가능
성과: 포트폴리오를 통해 면접 요청 2건 수락
이커머스 웹 애플리케이션
2.
기간: 2023년 1월 - 2023년 3월
기술 스택: Vue.js, Vuex, Firebase, Stripe API
설명: 팀 프로젝트로 이커머스 웹 애플리케이션 개발
주요 기여:
Vuex를 활용한 상태 관리 구현 및 효율적인 데이터 흐름 설계
Stripe API를 사용해 결제 기능 통합 및 테스트 환경 구축
제품 카테고리별 필터링 및 검색 기능 개발
Firebase Authentication을 활용한 사용자 인증 및 권한 관리 추가
성과: 프로젝트 완료 후 80% 이상의 코드 리뷰 반영
블로그 플랫폼 개발
3.
기간: 2022년 10월 - 2022년 12월
기술 스택: Next.js, Markdown, Tailwind CSS, Vercel
설명: Markdown 파일 기반의 정적 블로그 플랫폼 개발
주요 기여:
Next.js의 정적 사이트 생성(SSG) 기능을 활용하여 빠르고 SEO 친화적인 페이지
구현
Tailwind CSS를 사용하여 간결한 UI/UX 디자인 구축
다크 모드 및 커스텀 테마 옵션 추가
포스트 태그 관리 기능 및 태그 기반 검색 기능 구현
성과: 개인 블로그 운영으로 월간 방문자 500명 기록
실시간 채팅 애플리케이션
4.
프론트엔드 개발자 이력서 2기간: 2022년 6월 - 2022년 8월
기술 스택: React,

In [70]:
# !pip install transformers torch

In [71]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained("ilsilfverskiold/tech-keywords-extractor")
model = AutoModelForSeq2SeqLM.from_pretrained("ilsilfverskiold/tech-keywords-extractor")

keywords = ""
# 입력 텍스트를 토큰화하고 모델에 입력
for value in project_chunks:
    inputs = tokenizer.encode("extract tech keywords: " + value, return_tensors="pt", max_length=1000, truncation=True)
    outputs = model.generate(inputs, max_length=150, num_beams=2, early_stopping=True)
    keywords += tokenizer.decode(outputs[0], skip_special_tokens=True)

# 결과 디코딩
print(keywords)

Vue.js, Vue, Firebase, GitHub, Stripe API, React, Stripe, AuthenticationFirestore, React, React Context API, Google, SPA, Firebase


In [64]:
type(keywords)

str

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained("ilsilfverskiold/tech-keywords-extractor")
model = AutoModelForSeq2SeqLM.from_pretrained("ilsilfverskiold/tech-keywords-extractor")

# 입력 텍스트를 토큰화하고 모델에 입력
inputs = tokenizer.encode("extract tech keywords: " + project_section, return_tensors="pt", max_length=1000, truncation=True)
outputs = model.generate(inputs, max_length=150, num_beams=2, early_stopping=True)

# 결과 디코딩
keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(keywords)